# RAG Fractions – Notebook

Ce notebook **construit / charge** un vectorstore FAISS à partir du dossier `data/corpus/` puis teste la récupération et les réponses.

> Prérequis: avoir un `.env` avec `OPENAI_API_KEY` et un corpus dans `data/corpus/`.

In [ ]:
# ✅ 1) Imports & configuration
from pathlib import Path
import os

from dotenv import load_dotenv
load_dotenv()

assert os.getenv('OPENAI_API_KEY'), 'OPENAI_API_KEY manquant dans .env'

# Répertoires (conformes au projet)
BASE_DIR = Path.cwd()
DOCS_DIR = Path(os.getenv('DOCS_DIR', str(BASE_DIR / 'data' / 'corpus')))
PERSIST_PATH = Path(os.getenv('PERSIST_PATH', str(BASE_DIR / 'vectorstore' / 'faiss_store')))

print('DOCS_DIR:', DOCS_DIR)
print('PERSIST_PATH:', PERSIST_PATH)


In [ ]:
# ✅ 2) Vérifier le corpus
import glob

pdfs = sorted(glob.glob(str(DOCS_DIR / '**/*.pdf'), recursive=True))
txts = sorted(glob.glob(str(DOCS_DIR / '**/*.txt'), recursive=True))
print('PDF:', len(pdfs))
print('TXT:', len(txts))

for p in (pdfs + txts)[:10]:
    print('-', Path(p).relative_to(DOCS_DIR))


## 3) Charger le RAG depuis `rag_langchain.py`

On importe `rag_chain` (LangChain) tel qu’il est utilisé par Chainlit.

In [ ]:
from rag_langchain import rag_chain

rag_chain

## 4) Test simple question → réponse

In [ ]:
question = "Pourquoi certains élèves font 1/2 + 1/3 = 2/5 ?"
result = rag_chain({"question": question})

print('ANSWER:\n', result.get('answer',''))
print('\nN_SOURCES:', len(result.get('source_documents', [])))


## 5) Inspecter les sources récupérées

In [ ]:
sources = result.get('source_documents', [])
for i, d in enumerate(sources, 1):
    meta = d.metadata or {}
    src = meta.get('source', meta.get('basename', 'unknown'))
    page = meta.get('page', None)
    print(f"[{i}] {Path(src).name} | page={page}")
    print(d.page_content[:400].replace('\n',' '))
    print('---')


## 6) Mini-batterie de tests

Adapte les questions au contenu réel de ton corpus.

In [ ]:
questions = [
    "Comment comparer 3/7 et 4/7 ?",
    "Comment simplifier 12/18 ?",
    "Quelle erreur fréquente quand on additionne des fractions ?",
]

for q in questions:
    r = rag_chain({"question": q})
    print("Q:", q)
    print("A:", r.get('answer','')[:600])
    print("-"*80)
